In [ ]:
# Definindo a paleta de cores
AZUL1, AZUL2, AZUL3, AZUL4, AZUL5 = '#03045e', '#0077b6', "#00b4d8", '#90e0ef', '#CDDBF3'
CINZA1, CINZA2, CINZA3, CINZA4, CINZA5 = '#212529', '#495057', '#adb5bd', '#dee2e6', '#f8f9fa'
VERMELHO1, LARANJA1, AMARELO1, VERDE1, VERDE2 = '#e76f51', '#f4a261',	'#e9c46a', '#4c956c', '#2a9d8f'


In [ ]:
# Importanod bibliotecas
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt
import os
import seaborn as sns
import plotly.express as px
from matplotlib import transforms
import locale 


In [ ]:
# Setando o local para a formatação dos valores para dinheiro
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

In [ ]:
# Importando base vendas
file_relatorio_vendas = 'relatorio_vendas.csv'
df_vendas = pd.read_csv(file_relatorio_vendas)
display(df_vendas)
df_vendas.info()

In [ ]:
# formatando as colunas de data para datetime 
df_vendas['data_envio'] = pd.to_datetime(df_vendas['data_envio'], format="%Y-%m-%d")
df_vendas['data_pedido'] = pd.to_datetime(df_vendas['data_pedido'], format='%Y-%m-%d')
df_vendas = df_vendas.sort_values(by=['data_pedido','data_envio'], ascending=False)
display(df_vendas)
df_vendas.info()

In [ ]:
# criando uma variavel que receba a copia do dataframe de vendas
df_vendas_ano = df_vendas.copy()

#Pegando somente as colunas de data e vendas
df_vendas_ano = df_vendas_ano[['data_pedido', 'vendas']]

#Pegando somente o ano da data e atirbuindo a uma coluna ano
df_vendas_ano["ano"] = df_vendas_ano.data_pedido.dt.year

# Dropando a coluna que nao sera mais usada
df_vendas_ano.drop(columns='data_pedido', axis=1, inplace=True)

#Agrupando o total das vendas pelo ano 
df_vendas_ano = df_vendas_ano.groupby(by='ano').sum()

#df_vendas_ano['vendas'] = df_vendas_ano['vendas'].map(lambda vendas:locale.currency(vendas, grouping=True))
display(df_vendas_ano)
df_vendas_ano.info()

In [ ]:
#Criando uma função que gera grafico 
def grafico_vendas(cores: list=[AZUL2]):
    fig, ax = plt.subplots(figsize=(15,5))
    sns.set_theme(style='white')
    ax = sns.barplot(data=df_vendas_ano, x=df_vendas_ano.index, y='vendas', palette=cores)
    ax.set_title('Variação das vendas entre 2016 e 2019', loc='left', fontsize=18, color=CINZA1)
    ax.set_xlabel('Anos')
    ax.xaxis.set_tick_params(labelsize=14, labelcolor=CINZA2)
    sns.despine(left=True, bottom=True)
    ax.set_yticklabels([])
    fator_divisor = 1000
    for i, v in enumerate(df_vendas_ano['vendas']):
        #ax.text(v/fator_divisor, i, str(v), fontsize=2, va='center', color=CINZA2)
        qtd = f'R${v:,.0f}'.replace(',','.')
        offset = 1e5
        ax.text(i,v + offset, qtd, color=CINZA2, fontsize=12, ha ='center', va='center')
        
    
    fig.show()

    
    return ax

ax= grafico_vendas()

In [ ]:
# cria um laço para determinar qual cor é colocada na lista a partir da condição
cores = []

for ano in df_vendas_ano.index:
    if df_vendas_ano.loc[ano, 'vendas' ] == df_vendas_ano.vendas.max():
        cores.append(AZUL2)
    else:
        cores.append(AZUL5)
        
grafico_vendas(cores)

In [ ]:
# Dataframe com tipo de produto e lucro
top_lucro = df_vendas.copy()
top_lucro =top_lucro[['tipo_produto','lucro']]
top_lucro = top_lucro.groupby(by='tipo_produto').sum().sort_values(by='lucro',ascending=False)
top_7 = top_lucro[:7]
display(top_7)

#top_lucro= top_lucro['lucro'].map(lambda produto : locale.currency(produto, grouping=True))
#top_lucro = pd.DataFrame(top_lucro)

In [ ]:
## Gerando a função para criar o gráfico de barras

def grafico_top_7():
  
# Área do gráfico e tema da visualização
  fig, ax = plt.subplots(figsize=(10,4))
  fig.subplots_adjust(right=0.7)
  sns.set_theme(style="white")

# Definindo as cores do gráfico
  cores = [AZUL2, AZUL2, AZUL2, VERDE1, VERDE1, CINZA2, AMARELO1]

# Gerando o gráfico de barras
  ax = sns.barplot(data = top_7, x="lucro", y = top_7.index, palette = cores)
  
# Personalizando o gráfico
  ax.set_title('Top 7 produtos com maior lucro no catálogo (2016-2019)\n', fontsize=18, color=CINZA1, loc='left')
  ax.set_xlabel('')
  ax.set_ylabel('')
  ax.set_xticklabels([])
  ax.yaxis.set_tick_params(labelsize=10, labelcolor = CINZA2)
  sns.despine(left = True, bottom = True)

  # Escrevendo os valores de cada barra no gráfico
  for i, valor in enumerate(top_7['lucro']):
      qtd = f'R$ {valor:,.0f}'.replace(',','.')
      offset = 1e3  # offset de 1.000
      ax.text(valor - offset, i, qtd, color= CINZA5, fontsize=10, fontweight='bold', ha='right', va='center')
      
  return ax

ax = grafico_top_7()


In [ ]:

def texto_colorido(x, y, texto, cores, esp=20, ax=None, **kw):
    cores = list(reversed(cores))
    t = ax.transData
    canvas = ax.figure.canvas

    for i, linha in enumerate(reversed(texto.split('\n'))):
        frases = linha.split('||')
        for s, cor in zip(frases, cores[i]):
            texto = ax.text(x, y, s, color=cor, transform=t, **kw)
            texto.draw(canvas.get_renderer())
            ex = texto.get_window_extent()
            t = transforms.offset_copy(texto._transform, x=ex.width,
                                       units='dots')

        t = transforms.offset_copy(ax.transData, x=0, y=(i + 1) * esp, units='dots')

## Configurando o gráfico com parâmetros que potencializam a visualização dos dados

# Chamando a função do gráfico de barras
ax = grafico_top_7()

# Anotando uma conclusão no gráfico
texto_colorido(
    9.2e4, 3.25,    # coordenadas
    'Os dados indicam que os 3 produtos que geram\n'# texto
    '$\\bf{maior\ lucro}$|| são do departamento ||$\\bf{Automotivo}$.\n'
    '\n'
    'Podemos notar também que o departamento de\n'
    '$\\bf{Jardinagem\ e\ paisagismo}$|| possui 3 produtos com\n'
    'uma boa margem de lucro, sendo que um deles está\n'
    'abaixo de ||$\\bf{50\ mil\ reais}$|| no período mencionado.',
    
    [[CINZA2],# linha 1 # cores
     [CINZA1, CINZA2, AZUL2], # linha 2
     [CINZA2],# linha 3
     [CINZA2],# linha 4
     [VERDE1, CINZA2],# linha 5
     [CINZA2],# linha 6
     [CINZA2, CINZA1, CINZA2]# linha 7
    ],
    esp=22,	# espaçamento
    ax=ax, # figura onde desenhar o texto
    fontsize=10)

fig = ax.get_figure()


In [ ]:
# Dataframe vendas por ano
vendas_ano = df_vendas.copy()
vendas_ano = vendas_ano[['lucro', 'data_pedido']]
vendas_ano['ano'] = vendas_ano['data_pedido'].dt.year
vendas_ano = vendas_ano.drop(columns='data_pedido', axis=1)
vendas_ano =  vendas_ano.groupby(by='ano').sum()
vendas_ano = vendas_ano.sort_values(by='lucro')
display(vendas_ano)

In [ ]:
def gerar_grafico_lucro_ano():
    # Area da figura 
    fig, axs = plt.subplots(figsize=(14,5))
    sns.set_theme(style='white')
    fig.subplots_adjust(right=0.8) 

    # Plotando o gráfico
    axs = sns.barplot(data=vendas_ano, x='ano', y='lucro', palette=cores)
    axs.set_title('Lucro das vendas por ano', loc='left', fontsize=16, color=CINZA1)
    axs.set_xlabel('')
    axs.set_ylabel('')
    axs.set_yticklabels('')
    sns.despine(left=True, bottom=True)

    for i, v in enumerate(vendas_ano['lucro']):
        # Formatando a saida do valor
        qtd = f'R${v:,.0f}'.replace(',','.')
        offset = 1e4
        axs.text(i,v + offset, qtd, color=CINZA1, fontsize=12, ha='center', va='center')    
    return 
gerar_grafico_lucro_ano()

In [ ]:
# Dataframe gerando top10 
top_10 = df_vendas.copy()
top_10 = top_10[['vendas','data_pedido']]
top_10['ano'] = top_10['data_pedido'].dt.year
top_10 = top_10.drop(columns='data_pedido')
top_10 = top_10.groupby(by='ano').sum()
display(top_10)

In [ ]:
def gerar_grafico_top10():
    fig, ax = plt.subplots(figsize=(10,4))
    sns.set_theme(style="white")
    fig.subplots_adjust(right=0.7)

    ax = sns.barplot(data=top_10, x=top_10.index, y='vendas', palette=cores)
    ax.set_title('Lucro das vendas por ano', loc='left', fontsize=16, color=CINZA1)
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.set_yticklabels('')
    sns.despine(left=True, bottom=True)


    for i, v in enumerate(top_10['vendas']):
        # Formatando a saida do valor
        qtd = f'R${v:,.0f}'.replace(',','.')
        offset = 1e5
        ax.text(i,v + offset, qtd, color=CINZA1, fontsize=12, ha='center', va='center')    
    

    ax.text(3.5, 9.5e5,
         'O maior lucro registrado foi no ano $\\bf{2019}$.\n'
         'Em comparação com 2018, o lucro\n'
         'subiu aproximadamente $\\bf{14,04}$%.',
         fontsize=10, 
         linespacing=1.15, 
         color=CINZA1)

    return

gerar_grafico_top10()


In [ ]:
# Área do gráfico e tema da visualização
fig, ax = plt.subplots(figsize=(10,4))
fig.subplots_adjust(right=0.7)
sns.set_theme(style="white")

# Definindo as cores do gráfico: 3 primeiro em azul e restante em cinza 
cores = [AZUL2 if i < 3 else CINZA3 for i in range(10)]

# Gerando o gráfico de barras 
ax = sns.barplot(data=top_10, x=top_10.index, y='vendas', palette = cores)

# Personalizando o gráfico
ax.set_title('Top 10 produtos com maiores vendas no catálogo (2016-2019)\n', fontsize=18, color=CINZA1, loc='left')
ax.set_xlabel('')
ax.set_ylabel('')
ax.set_xticklabels([])
ax.yaxis.set_tick_params(labelsize=10, labelcolor = CINZA2)
sns.despine(left = True, bottom = True)

# Escrevendo os valores de cada barra no gráfico
for i, valor in enumerate(top_10['vendas']):
    qtd = f'R$ {valor:,.0f}'.replace(',','.')  
    offset = 1e7  # offset de 10.000
    ax.text(valor - offset, i, qtd, color= CINZA5, fontsize=8, fontweight='bold', ha='right', va='center')


# Gerando o texto colorido
texto_colorido(
    1.05e6, 4,                           # coordenadas
    'Os dados indicam que os 3 produtos que || $\\bf{mais}$\n'
    'venderam foram:|| $\\bf{pneus,ferramentas e vasos}$.\n'
    '\n'
    'Podemos notar no período mencionado também que as\n'
    'ferramentas de jardinagem e materiais de revestimento\n' 
    'também possuem uma boa saída.',
    [[CINZA2, CINZA1],                # linha 1 # cores
     [CINZA2, AZUL2],                 # linha 2
     [CINZA2],                        # linha 3
     [CINZA2],                        # linha 4
     [CINZA2],                        # linha 6
     [CINZA2]
    ],
    esp=22,         # espaçamento
    ax=ax,          # figura onde desenhar o texto
    fontsize=10)    


# Exibindo o gráfico
plt.show()


In [ ]:
# Vendas por ano e região
ano_regiao = df_vendas.copy()
ano_regiao = ano_regiao[['data_pedido','vendas', 'regiao']]
ano_regiao['ano'] = ano_regiao.data_pedido.dt.year
ano_regiao.drop(labels='data_pedido', axis=1, inplace=True)

df_ano_regiao = pd.crosstab(index= ano_regiao.ano, columns=ano_regiao.regiao, values=ano_regiao.vendas, aggfunc='sum')
df_ano_regiao = df_ano_regiao[['Sudeste', 'Nordeste', 'Centro-Oeste','Norte', 'Sul']]
display(df_ano_regiao)


# df_ano_regiao = ano_regiao.groupby(by=['ano','regiao']).sum()

In [ ]:
def gera_grafico_ano_regiao():

    cores = [AZUL1, AZUL2, AZUL3,AZUL4,CINZA3]
    
    ax = df_ano_regiao.plot(kind='bar', stacked=True, figsize=(14,8), color=cores)
    ax.set_title('Vendas por ano e região', fontsize=30, color=AZUL1, loc='center')
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.set_yticklabels([])
    ax.xaxis.set_tick_params(labelsize='14', labelcolor=CINZA1, rotation=0)
    ax.set_frame_on(False)

#Removendo todos os ticks possiveis do exio x e y
    ax.tick_params(axis='both', which='both', length=0)


#Conclusao do grafico

    ax.text(3.55, 1e6,'87%',fontweight='bold', fontsize=25, color=AZUL2)
    ax.text(3.55, 5.5e5, 
             
             'das vendas nas lojas\n '
             'estão concentradas nas\n'
             'regiões $\\bf {Sudeste}$, $\\bf{Nordeste}$\n'
             'e $\\bf{Centro-Oeste}$ em 2019.',
             fontsize=14,
             linespacing=1.60,
             color=AZUL1
             )

    return ax
ax = gera_grafico_ano_regiao()

ax.legend(bbox_to_anchor=(1,1), reverse=True, title='Região',title_fontsize= 12,  fontsize=12 )

for container in ax.containers:
    labels=[f'R${valor.get_height()/1000:.0f}K'.replace(',','.') for valor in container]
    ax.bar_label(container, label_type='center', labels = labels, size=10, color=CINZA5, fontweight='bold')
    
ax.axvline(x=3.3, ymin=0, ymax=0.83,color= AZUL1, linewidth=1.2)


In [ ]:
df_envio = df_vendas.copy()
df_envio = df_envio[['modo_envio', 'segmento_cliente']]
df_envio_seg = pd.crosstab(index=df_envio.modo_envio, columns=df_envio.segmento_cliente).sort_values('B2B')

# NORMALIZE PELO INDEX
df_envio_normalizado = pd.crosstab(index=df_envio.modo_envio, columns=df_envio.segmento_cliente, normalize='index').sort_values('B2B')
display(df_envio_normalizado)

In [ ]:
# Função para escrita de texto utilizando diferentes cores. Não existe uma 
# função nativa para isso. (Adaptado de https://github.com/empathy87/storytelling-with-data)
from matplotlib import transforms

def texto_colorido(x, y, texto, cores, esp=20, ax=None, **kw):
    cores = list(reversed(cores))
    t = ax.transData
    canvas = ax.figure.canvas

    for i, linha in enumerate(reversed(texto.split('\n'))):
        frases = linha.split('||')
        for s, cor in zip(frases, cores[i]):
            texto = ax.text(x, y, s, color=cor, transform=t, **kw)
            texto.draw(canvas.get_renderer())
            ex = texto.get_window_extent()
            t = transforms.offset_copy(texto._transform, x=ex.width, 
                                       units='dots')

        t = transforms.offset_copy(ax.transData, x=0, y=(i + 1) * esp, units='dots')


In [ ]:
# Área do gráfico e tema da visualização
fig, axs = plt.subplots(1, 2, figsize=(12,6), sharey = True)
# Definindo as cores do gráfico
cores = [AZUL2, AZUL5]

# GRÁFICO 1 - Barras empilhadas
axs[0].barh(df_envio_seg.index, df_envio_seg["B2B"], color = cores[0])
axs[0].barh(df_envio_seg.index, df_envio_seg["B2B"], color = cores[0])
axs[0].barh(df_envio_seg.index, df_envio_seg["B2C"], color = cores[0])
axs[0].barh(df_envio_seg.index, df_envio_seg["B2C"], left = df_envio_seg["B2B"], color = cores[1])
axs[0].barh(df_envio_seg.index, df_envio_seg["B2B"], color = cores[0])
axs[0].barh(df_envio_seg.index, df_envio_seg["B2C"], left = df_envio_seg["B2B"], color = cores[1])
axs[0].set_title('Em valores absolutos\n', color = CINZA2)
axs[0].set_xlabel('')
axs[0].set_ylabel('')
axs[0].set_xticklabels([])
axs[0].yaxis.set_tick_params(labelsize=14, color = CINZA2)
axs[0].set_frame_on(False)

# remover todos os ticks do eixo x e y
axs[0].tick_params(axis='both', which='both', length=0)

texto_colorido(0, 3.5, '$\\bf{B2B}$ | || $\\bf{B2C}$', cores = [cores], ax=axs[0], fontsize=12) # legenda


# GRÁFICO 2 - Barras empilhadas 100 %
axs[1].barh(df_envio_normalizado.index, df_envio_normalizado["B2B"], color = cores[0])
axs[1].barh(df_envio_normalizado.index, df_envio_normalizado["B2C"], left = df_envio_normalizado["B2B"], color = cores[1])
axs[1].set_title('Em porcentagem\n', color = CINZA2)
axs[1].set_xlabel('100%', color = CINZA2, size = 14, loc='right')
axs[1].set_ylabel('')
axs[1].set_xticklabels([])
axs[1].set_frame_on(False)

# remover todos os ticks do eixo x e y
axs[1].tick_params(axis='both', which='both', length=0)

texto_colorido(0, 3.5, '$\\bf{B2B}$ | || $\\bf{B2C}$', cores = [cores], ax=axs[1], fontsize=12) # legenda


# Título dos gráficos
fig.suptitle("Modo de envio mais utilizado pelos clientes B2B/B2C (2016-2019)\n", 
             fontsize=18, color=CINZA1, ha = 'right', x = 0.95, y = 1.05)


# Valores das barras
for container in axs[0].containers:
    labels = [str(valor.get_width()) if valor.get_width() > 300 else '' for valor in container]
    axs[0].bar_label(container, label_type='center', labels = labels, size = 12, color = CINZA5, fontweight='bold')

for container in axs[1].containers:
    labels = [f'{valor.get_width()*100:.1f}%' for valor in container]
    axs[1].bar_label(container, label_type='center', labels = labels, size = 12, color = CINZA5, fontweight='bold')

fig.show()



In [ ]:
# Dataframe por regiaoe trimestre
df_sp = df_vendas.copy()
df_sp = df_sp.query('estado == "São Paulo"')[['data_pedido','vendas']].sort_values(by='data_pedido')

#agrupando por trimestre 
df_sp.set_index('data_pedido', inplace=True)
df_sp = df_sp.resample('Q').agg('sum')
df_sp['vendas'] = round(df_sp['vendas']/ 1e3, 2)
df_sp = df_sp.reset_index()

display(df_sp)

In [ ]:
import matplotlib.dates as mdates

In [ ]:
def gerar_grafico_sp_tri():
    
    fig, ax = plt.subplots(figsize=(11,4))
    
    #grafico de linha
    pontos_a_marcar = [True if mes ==12 else False for mes in df_sp['data_pedido'].dt.month]
    
    ax.plot(df_sp['data_pedido'], df_sp['vendas'], lw=4, color= AZUL1, marker='o', markersize=10, markerfacecolor=AZUL4, markevery = pontos_a_marcar)

    
    
    ax.set_title('Gráfico de vendas trimestral do estdo de São Paulo', fontsize=18, color=CINZA1, loc='left', fontweight='bold')
    ax.text(0.5, 0.95,'De 2016 ate 22019 (em milhares de reais)', transform = ax.transAxes, color =CINZA2, fontsize=14, ha='right', va='center')
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.set_yticklabels([])
    ax.set_frame_on(False)
    ax.xaxis.set_tick_params(labelsize=14, labelcolor=CINZA1)
    ax.tick_params(axis='both',which='both',length=0)
    ax.grid(axis='y',linestyle='--')

    
    ax.xaxis.set_major_locator(mdates.MonthLocator(interval = 3))
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))
    x_labels = ['mar', 'jun', 'set','dez'] * 4
    ax.set_xticks(df_sp.data_pedido)
    ax.set_xticklabels(x_labels, ha='center', ) 

    texto = [16*" " + str(i) + 16*" " for i in range(2016, 2020)]
    ax.text(ax.get_xbound()[0] + 35, - 80, f'|{texto[0]}|{texto[1]}|{texto[2]}|{texto[3]}|', fontsize=14, color=CINZA2)

    
    
    ano = 0
    for i in range(3, len(df_sp["vendas"]), 4):
        ax.text(x = ax.get_xticks()[i], y = df_sp["vendas"][i] + 20, s = f"R$ {df_sp['vendas'][i]} mil", color = AZUL2, weight = "bold", fontsize = 10)
        ax.text(x = ax.get_xticks()[i] + 160, y = df_sp["vendas"][i] + 20, s = f"(4º Tri {2016 + ano})", color = CINZA2, weight = "bold", fontsize = 10)
        ano +=1
    
    plt.ylim(0,400)
    return ax

ax = gerar_grafico_sp_tri()

In [ ]:
#criando um df de regiao que contenha uma deep cpoy do df vendas
df_regiao = df_vendas.copy()
df_regiao = df_regiao[['regiao','data_pedido','vendas']] #recebe apenas essas colunas selecionadas

#realizando o cross tab pois teremos duas variaveis nesse grafico vendass por trimestre e regiao
df_regiao = pd.crosstab(index=df_regiao.data_pedido, columns=df_regiao.regiao, values=df_regiao.vendas, aggfunc='sum')

#organizando o df regiao em ordem de maior para o menor
df_regiao = df_regiao[["Sudeste", "Nordeste", "Centro-Oeste", "Norte", "Sul"]]

#Realizando a agreagação utilizando o resample por trimestre e somando
df_regiao = df_regiao.resample('Q').agg('sum')

#formatando o df para ter valores das vendas apenas com duas casas decimais e dividindo o valor por mil     
df_regiao= round(df_regiao/ 1e3, 2)

display(df_regiao)

In [ ]:
import plotly.express as    px

In [133]:
def gerar_grafico_tri_dinamico():

    fig = px.line(data_frame=df_regiao,
                  x=df_regiao.index,
                  y=df_regiao.columns,
                  markers=True,
                  labels={'regiao':'Região'},
                  color_discrete_sequence=[AZUL2, VERMELHO1,AMARELO1,VERDE1, CINZA3])
    
    fig.update_layout(
        width=1300, 
        height=600, 
        font_family = 'DejaVu Sans', 
        font_size=15, 
        font_color= CINZA2, 
        title_font_color= CINZA1, 
        title_font_size=24,
        title_text='Faturamento da loja de departamentos por trimestre' + '<br><sup size=1 style="color:#555655">De 2016 a 2019</sup>',
        xaxis_title='', 
        yaxis_title='', 
        plot_bgcolor= CINZA5)
    
    fig.update_yaxes(tickprefix="R$ ", ticksuffix=" mil")

    labels = ['T1<br>2016', 'T2<br>2016', 'T3<br>2016', 'T4<br>2016',
           'T1<br>2017', 'T2<br>2017', 'T3<br>2017', 'T4<br>2017',
           'T1<br>2018', 'T2<br>2018', 'T3<br>2018', 'T4<br>2018',
           'T1<br>2019', 'T2<br>2019', 'T3<br>2019', 'T4<br>2019']
    fig.update_xaxes(ticktext = labels, tickvals=df_regiao.index)
    fig.update_traces(mode='markers+lines', hovertemplate ='<b>Período<b> %{x} br <b>Faturamento<b> %{y}')
    fig.update_layout(hovermode='closest')    
    fig.update_layout(hovermode='x')          
    fig.update_layout(hovermode='x unified')  


    return fig

grafico = gerar_grafico_tri_dinamico()

display(grafico)